# 3 Adding Parameters and Fields
***
***
So far, ActiveRegion has a set of in built functions. If you want to add another feature or field, this guide will show you how to do that.

In [1]:
from flares.active_region import ActiveRegion
from flares.data import get_dates
import torch
import numpy as np

### First, create an active region subclass

In [2]:
class ActiveRegionDeriv(ActiveRegion):
    
    def __init__(self, hnum, date, root):
                
        # Create an active region - note the 3 at the end is the 3 extra parameters you are adding
        super().__init__(hnum, date, root, 3)
        
        """
        We want to register two new functions:
        
        num_in_mask - returns number of true values in a mask and number of false values in a mask (2 parameters returned)
        num_hc_gt_zero - returns number of pixels in hc that are greater than 0 (1 parameter returned)
        
        Simply call register func to the two parameter functions that return the values you want
        """
        self.register_func(self.num_in_mask)
        self.register_func(self.num_hc_gt_zero)

        
    def num_in_mask(self, mask):
        """
        Mask is a torch array the same size as Bz Bx By and cont (self.shape)
        
        It is a boolean array with true values on the subset of the active region
        that ActiveRegion is interested in.
        
        A feature function returns a label followed by a scalar (or list of labels and scalars)
        """
        return ["num_1_in_mask", "num_0_in_mask"], [torch.count_nonzero(mask), torch.count_nonzero(~mask)]
    
    
    def num_hc_gt_zero(self, mask):
        """
        Count the number of values in hc greater than 0
        """
        self.assert_hc_gt_zero()
        return "num_hc_gt_zero", torch.count_nonzero(self.hc_gt_zero)
    
    
    # An example of another field (this is a pretty bad design because we could just count number positive in the parameter function,
    # but this is just an example)
    def assert_hc_gt_zero(self):
        """
        Creating a new field is relatively easy, just check if the field already exists, then
        define the field. In this case, we create a new field called hc_gt_zero (1 if hc is gt 0 0 else)
        and we need the hc field first. All of these derived fields are torch arrays
        """
        if not hasattr(self, "hc_gt_zero"):
            self.assert_hc() # We need the hc field as a prereq for gt_zero
            self.hc_gt_zero = (self.hc > 0).double()

### Test the active region with the three added features:
num_hc_gt_zero, num_0_in_mask, num_1_in_mask

In [3]:
hnum = 7115
hnum = 7115 
root = "/srv/data/thli2739"
dates = get_dates(hnum, root, sort = True) # If you don't want to manually put in dates - extract all possible dates

ar = ActiveRegionDeriv(hnum, dates[0], root)

In [4]:
data, labels = ar.get_baseline()
print("Baseline new features: ", data[-3:], labels[-3:], end = "\n\n")

data, labels = ar.get_segmented()
print("Segmented new features: ", data[-3:], labels[-3:], end = "\n\n")

print("Graph: ")
G, labels = ar.get_graph()
for n, data in G.nodes.data():
    print("Node: ", n, "Type: ", data["type"], data["v"][-3:], labels[-3:])

Baseline new features:  [308224.      0. 153797.] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']

Segmented new features:  [303684.   4540. 153797.] ['bckg_num_1_in_mask', 'bckg_num_0_in_mask', 'bckg_num_hc_gt_zero']

Graph: 
Node:  0 Type:  penumbra [   433. 307791. 153797.] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']
Node:  1 Type:  umbra [7.00000e+01 3.08154e+05 1.53797e+05] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']
Node:  2 Type:  umbra [4.50000e+01 3.08179e+05 1.53797e+05] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']
Node:  3 Type:  penumbra [  1498. 306726. 153797.] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']
Node:  4 Type:  umbra [   370. 307854. 153797.] ['bas_num_1_in_mask', 'bas_num_0_in_mask', 'bas_num_hc_gt_zero']


## What about new fields (other than continuum / Bz)?
Here's a new field called random. We want to add this field to the fields in ActiveRegion

In [5]:
random_field = torch.rand(448, 688)

In [6]:
class ActiveRegionDeriv(ActiveRegion):
    
    
    def __init__(self, hnum, date, root):
                
        # Create an active region - note the 3 at the end is the 3 extra parameters you are adding
        super().__init__(hnum, date, root, 1)
        
        """
        We want to register one new function that uses another field (not Bz Bx By or cont)
        
        to access this new field, we would type: self.rand_field
        """
        self.manually_add_field(random_field, "rand_field")
        self.register_func(self.magnitude)

        
    def magnitude(self, mask):
        """
        Get the sum of the random field
        when we use new fields, make sure they're torch tensors on the right device
        """
        return "rand_sum", torch.sum(self.rand_field[mask])

In [7]:
hnum = 7115
hnum = 7115 
root = "/srv/data/thli2739"
dates = get_dates(hnum, root, sort = True) # If you don't want to manually put in dates - extract all possible dates

ar = ActiveRegionDeriv(hnum, dates[0], root)

In [8]:
data, labels = ar.get_baseline()
print("Baseline new features: ", data[-1:], labels[-1:], end = "\n\n")

data, labels = ar.get_segmented()
print("Segmented new features: ", data[-1:], labels[-1:], end = "\n\n")

print("Graph: ")
G, labels = ar.get_graph()
for n, data in G.nodes.data():
    print("Node: ", n, "Type: ", data["type"], data["v"][-1:], labels[-1:])

Baseline new features:  [154265.96875] ['bas_rand_sum']

Segmented new features:  [151966.6875] ['bckg_rand_sum']

Graph: 
Node:  0 Type:  penumbra [215.94555664] ['bas_rand_sum']
Node:  1 Type:  umbra [36.23317337] ['bas_rand_sum']
Node:  2 Type:  umbra [21.79161072] ['bas_rand_sum']
Node:  3 Type:  penumbra [764.59313965] ['bas_rand_sum']
Node:  4 Type:  umbra [195.72195435] ['bas_rand_sum']
